# Dynamically returning directly

Here LLM can optionally decide to return the result of a tool call as the final answer. This is useful in cases where you have tools that ca sometimes generate responses that are acceptable as final answers , and you want the llm to determine when that is the case


In [53]:
!pip install langchain
!pip install langchain-google-genai
!pip install google-generativeai
!pip install tavily-python
!pip install langchain-community

In [54]:
import os 
import getpass

In [55]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Google API Key")
os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API Key")

Google API Key ········
Tavily API Key ········


Optionally we can set API key for Langsmith tracing which will give best in class observability 

In [56]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Langsmith API Key")

Langsmith API Key ········


## Set up the tools 

In [61]:
!pip install -U langchain
!pip install -U langchain-community
!pip install -U langchain-community-tools


ERROR: Could not find a version that satisfies the requirement langchain-community-tools (from versions: none)
ERROR: No matching distribution found for langchain-community-tools


**Modification**

In [62]:
from langchain_core.pydantic_v1 import BaseModel, Field

class SearchTool(BaseModel):
    """Look Up things online , optionally returning directly"""
    query: str= Field(description = "query to look up online")
    return_direct: bool = Field(
        description = "Whether or not the result of this should be returned directly to the user without you seeing what it is",
        default = False
    )

In [63]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Initialize the TavilySearchResults tool with max_results set to 1
search_tool = TavilySearchResults(max_results=1 , args_schema = SearchTool)
tools = [search_tool]

Wrap these tools in a simple ToolExecutor . This is  as real simple class that takes in a  ToolInvocation and calls that tool , returning the output. A ToolInnvocation is any class with tool and tool_input attribute

In [64]:
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tools)

C:\Users\VISIONARY\AppData\Local\Temp\ipykernel_8620\780962383.py:3: LangGraphDeprecationWarning: ToolExecutor is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_executor = ToolExecutor(tools)


 ## Setup the model 

In [65]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools.tavily_search import TavilySearchResults



In [66]:
%pip install -qU langchain-groq

# Other necessary libraries

# Required for tools formatted to OpenAI's function calling structure
!pip install openai  
!pip install requests  

Note: you may need to restart the kernel to use updated packages.


In [67]:
import getpass
import os

os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

Enter your Groq API key:  ········


In [75]:
!pip install groq-api

ERROR: Could not find a version that satisfies the requirement groq-api (from versions: none)
ERROR: No matching distribution found for groq-api


In [76]:
from langchain.tools.render import format_tool_to_openai_function
from langchain_groq import ChatGroq

model = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    streaming = True
    # other params...
)

After this , we should make sure the model knows that it has these tools available to call. We can do this by converting the Langchain tools into the format for OpenAI function calling and then bind them to the model class

In [77]:

# Step 2: Define the tools and format them for function use
functions = [format_tool_to_openai_function(t) for t in tools]

# Step 4: Bind the functions to the model (adjust according to Groq API documentation)
model = model.bind_functions(functions)


## Define the agent state

The main type of graph in langgraph is the StatefulGraph. For this example, the state we will just be a list of messages. We want each node to just add messages to that list. Therefore , we shall use a TypedDict with one key (messages) and annotate it so that the messages attribute is always added to

In [78]:
from typing import TypedDict , Annotated, Sequence
import operator 
from langchain_core.messages import BaseMessage

class AgentState(TypedDict): 
    messages: Annotated[Sequence[BaseMessage], operator.add]

## Define the nodes 

we now need to define a few different nodes in our graph. In langgraph , a node can either be a function or a runnable.

Lets define the nodes , as well as a function to decide how what conditional edge to take

**Modification**

We change the should_continue function to check whether return_direct was set to True

In [79]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

# Define the function that determines whether to continue or not 
def should_continue(state):
    messages = state['messages']
    last_message = messages[-1]
    #if there is no function call 
    if 'function_call' not in last_message.additional_kwargs:
        return 'end'
    # otherwise if there is, we continue
    else:
        arguments = json.loads(last_message.additional_kwargs["function_call"]["arguments"])
        if arguments.get("return_direct", False):
            return "final"
        else:
            return 'continue'




In [80]:
# Define the function that calls the model
def call_model(state):
    messages = state['messages']
    response =model.invoke(messages)
    #we return a list , because this will get added to the existing list
    return {"messages": [response]}


**The modification**

We change the tool calling to get rid of the return_direct parameter 

In [88]:
#Define the function to execute tools
def call_tool(state):
    messages = state['messages']
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    #We construct a ToolInvocation from the functio|n_call 
    tool_name = last_message.additional_kwargs["function_call"]["name"]
    arguments = json.loads(last_message.additional_kwargs["function_call"]["arguments"])
    if tool_name == "tavily_search_results_json":
        if "return_direct" in arguments:
            del arguments["return_direct"]
    action = ToolInvocation(
        tool = tool_name,
        tool_input = arguments,
    )
    #we call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    #we use the response to create a FunctionMessage
    function_message = FunctionMessage(content =str(response), name = action.tool)
    #we return a list , because this will get added to the existing list
    return {"messages": [function_message]}

## Define the graph

We can now put it all together and define the graph

**Modification**

We add a separate node for any tool call where return_direct=True . The reason this is needed is that after this node we want to end, while after other tool calls we went to back to the llm

In [90]:
from langgraph.graph import StateGraph, END

# Define a new graph

workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between 
workflow.add_node("agent", call_model)
workflow.add_node("action" , call_tool)
workflow.add_node("final", call_tool)

#set entry point as 'agent'

workflow.set_entry_point("agent")

#we now add a conditional edge 
workflow.add_conditional_edges(
    #start node
    "agent",
    #function that determines which node is called next
    should_continue,
    # finally we pass a mapping
    # keys are strings and the values are the other nodes 
    #when should_continue is called , outpu will be matched against a keys in the mapping, 
    #based on which one it matches , that node will then be called
    {
        #if 'tools', then we call the tool node
        "continue": "action",
        #final call
        "final": "final",
        #otherwise
        "end": END
    }
)

# We now add a normal edge from 'tools' to 'agent'
#This means that after 'tools' is called, 'agent' node is called next

workflow.add_edge('action', 'agent')
workflow.add_edge('final', END)

#finally we compile . it compiles into a langchain runnale . 
# Can be used as like any runnable

app = workflow.compile()

## Use it 

we can now use it. it ow exposes the same interface as all other langchain runnables

In [92]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content = "what is the weather in sf")]}
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '(key)':")
        print("---")
        print(value)
    print("\n ---\n")

Output from node '(key)':
---
{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"weather in sf"}', 'name': 'tavily_search_results_json'}}, response_metadata={'finish_reason': 'function_call'}, id='run-c3523153-2ef5-41b6-b92a-daf3bff62851-0', usage_metadata={'input_tokens': 1289, 'output_tokens': 84, 'total_tokens': 1373})]}

 ---



C:\Users\VISIONARY\AppData\Local\Temp\ipykernel_8620\2285556600.py:13: LangGraphDeprecationWarning: ToolInvocation is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  action = ToolInvocation(


Output from node '(key)':
---
{'messages': [FunctionMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1724254951, \'localtime\': \'2024-08-21 08:42\'}, \'current\': {\'last_updated_epoch\': 1724254200, \'last_updated\': \'2024-08-21 08:30\', \'temp_c\': 17.2, \'temp_f\': 63.0, \'is_day\': 1, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/116.png\', \'code\': 1003}, \'wind_mph\': 8.1, \'wind_kph\': 13.0, \'wind_degree\': 250, \'wind_dir\': \'WSW\', \'pressure_mb\': 1018.0, \'pressure_in\': 30.07, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 84, \'cloud\': 75, \'feelslike_c\': 17.2, \'feelslike_f\': 63.0, \'windchill_c\': 14.8, \'windchill_f\': 58.7, \'heatindex_c\': 14.9, \'heatindex_f\': 58.9, \'dewpo

In [94]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content = "what is the weather  in sf? return this result directly by setting return_direct = True")]}
for output in app.stream(inputs):
    #stream () yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '(key)':")
        print("---")
        print(value)
    print("\n ---\n")

Output from node '(key)':
---
{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"weather in SF","return_direct":true}', 'name': 'tavily_search_results_json'}}, response_metadata={'finish_reason': 'function_call'}, id='run-9ed1983e-83d3-49a5-b263-34bd001ef135-0', usage_metadata={'input_tokens': 1302, 'output_tokens': 108, 'total_tokens': 1410})]}

 ---



C:\Users\VISIONARY\AppData\Local\Temp\ipykernel_8620\2285556600.py:13: LangGraphDeprecationWarning: ToolInvocation is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  action = ToolInvocation(


Output from node '(key)':
---
{'messages': [FunctionMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1724256974, \'localtime\': \'2024-08-21 09:16\'}, \'current\': {\'last_updated_epoch\': 1724256000, \'last_updated\': \'2024-08-21 09:00\', \'temp_c\': 17.8, \'temp_f\': 64.0, \'is_day\': 1, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/116.png\', \'code\': 1003}, \'wind_mph\': 2.2, \'wind_kph\': 3.6, \'wind_degree\': 10, \'wind_dir\': \'N\', \'pressure_mb\': 1019.0, \'pressure_in\': 30.08, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 81, \'cloud\': 75, \'feelslike_c\': 17.8, \'feelslike_f\': 64.0, \'windchill_c\': 15.7, \'windchill_f\': 60.2, \'heatindex_c\': 15.7, \'heatindex_f\': 60.3, \'dewpoint_